#### Challenge 44: DSA nonce recovery from repeated nonce

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)

In [1]:
from Crypto.Util import number
from Crypto.Random import random
from Crypto.Hash.SHA256 import SHA256Hash

import cryptopals as cp
import sha1

<div class="alert alert-block alert-info">
    
<div class="alert alert-block alert-warning">
    
#### **Cryptanalytic MVP award.**

This attack (in an elliptic curve group) broke the PS3. It is a great, great attack.
    
</div>

In [this file](challenge-data\44.txt) find a collection of DSA-signed messages. (NB: each msg has a trailing space.)

These were signed under the following pubkey:
```
y = 2d026f4bf30195ede3a088da85e398ef869611d0f68f07
    13d51c9c1a3a26c95105d915e2d8cdf26d056b86b8a7b8
    5519b1c23cc3ecdc6062650462e3063bd179c2a6581519
    f674a61f1d89a1fff27171ebc1b93d4dc57bceb7ae2430
    f98a6a4d83d8279ee65d71c1203d2c96d65ebbf7cce9d3
    2971c3de5084cce04a2e147821
```
(using the same domain parameters as the previous exercise)

It should not be hard to find the messages for which we have accidentally used a repeated "k". Given a pair of such messages, you can discover the "k" we used with the following formula:

$$ k = \frac{m1 - m2}{s1 - s2} \mod q $$

<br>    
<div class="alert alert-block alert-warning">

#### **9th Grade Math: Study It!**

If you want to demystify this, work out that equation from the original DSA equations.

</div>
    

<div class="alert alert-block alert-warning">
    
#### **Basic cyclic group math operations want to screw you**
    
Remember all this math is mod q; s2 may be larger than s1, for instance, which isn't a problem if you're doing the subtraction mod q. If you're like me, you'll definitely lose an hour to forgetting a paren or a mod q. (And don't forget that modular inverse function!)

</div>    

What's my private key? Its SHA-1 (from hex) is:   

`ca8f6f7c66fa362d40760d135b763eb8527d3d52`


</div>

In [2]:
p = 0x800000000000000089e1855218a0e7dac38136ffafa72eda7859f2171e25e65eac698c1702578b07dc2a1076da241c76c62d374d8389ea5aeffd3226a0530cc565f3bf6b50929139ebeac04f48c3c84afb796d61e5a4f9a8fda812ab59494232c7d2b4deb50aa18ee9e132bfa85ac4374d7f9091abc3d015efc871a584471bb1
q = 0xf4f47f05794b256174bba6e9b396a7707e563c5b
g = 0x5958c9d3898b224b12672c0b98e06c60df923cb8bc999d119458fef538b8fa4046c8db53039db620c094c9fa077ef389b5322a559946a71903f990f1f7e0e025e2d7f7cf494aff1a0470f5b64c36b625a097f1651fe775323556fe00b3608c887892878480e99041be601a62166ca6894bdd41a7054ec89f756ba9fc95302291

y = int(
    '2d026f4bf30195ede3a088da85e398ef869611d0f68f07'
    '13d51c9c1a3a26c95105d915e2d8cdf26d056b86b8a7b8'
    '5519b1c23cc3ecdc6062650462e3063bd179c2a6581519'
    'f674a61f1d89a1fff27171ebc1b93d4dc57bceb7ae2430'
    'f98a6a4d83d8279ee65d71c1203d2c96d65ebbf7cce9d3'
    '2971c3de5084cce04a2e147821', 16)

true_pk_fp = 'ca8f6f7c66fa362d40760d135b763eb8527d3d52'

---
Load message data from file and store it:

In [3]:
file_path = './challenge-data/44.txt'

msg_data = []

with open(file_path) as fp:    
    while True:
        msg = fp.readline()
        if not(msg):
            break
        msg = msg[5:].strip('\n')
        s = int(fp.readline()[3:].strip('\n'))
        r = int(fp.readline()[3:].strip('\n'))
        m = int(fp.readline()[3:].strip('\n'), 16)
        # Verify message hashes match.
        # I had to convert to ints for hash check because the text file 
        # hashes don't include leading 0's.
        assert(int(sha1.SHA1(msg.encode()).finish().hex(),16) == m)
        msg_data.append([msg, s, r, m])

In [4]:
def get_k(m1, s1, m2, s2):
    
    num = (m1 - m2) % q
    den = cp.invmod((s1 - s2) % q, q)
    return((num*den) % q)

In [12]:
key_found = False
for ii in range(len(msg_data)-1):
    
    for jj in range(ii, len(msg_data)):
        
        msg1 = msg_data[ii][0].encode()
        s1 = msg_data[ii][1]
        r1 = msg_data[ii][2]
        m1 = msg_data[ii][3]
        
        s2 = msg_data[jj][1]
        m2 = msg_data[jj][3]
        
        k = get_k(m1, s1, m2, s2)
        r_k = pow(g, k, p) % q
        if r_k == r1:
            key_found = True
            print('Found private key:\n')
            x = cp.dsa_priv_key_from_k(msg1, k, r1, s1, q)
            print(f'x = {hex(x)}')
            break
            
    if key_found:
        break

Found private key:

x = 0xf1b733db159c66bce071d21e044a48b0e4c1665a


In [9]:
x_bytes = hex(x)[2:].encode()
my_pk_fp = sha1.SHA1(x_bytes).finish().hex()

if my_pk_fp == true_pk_fp:
    print('Fingerprint matches')
else:
    print('Private key fingerprint does NOT match')

Fingerprint matches


<div class="alert alert-block alert-warning">

#### **9th Grade Math: Study It!**

If you want to demystify this, work out that equation from the original DSA equations.

</div>

How did we get to this?

$$ k = \frac{m1 - m2}{s1 - s2} \mod q $$


From Challenge 43:

<div class="alert alert-block alert-info">

Given a known `k`, it's trivial to recover the DSA private key `x`:

          (s * k) - H(msg)
      x = ----------------  mod q
                  r

</div>

Equating the private key recovery equation for two different messages with a shared `k`:


$$ \frac{(s_1 k) - h_1}{r_1} = \frac{(s_2 k) - h_2)}{r_2} $$

$$ \implies s_1 k - h_1 = \frac{r_1}{r_2} s_2 k - h_2$$

$$ \implies s_1 k - \frac{r_1}{r_2} s_2 k = h_1 - h_2 $$

$$ \implies k (s_1 - \frac{r_1}{r_2} s_2) = h_1 - h_2 $$

$$ \implies k = \frac{h_1 - h_2}{s_1 - \frac{r_1}{r_2} s_2} $$

Since $$ r = (g^k\mod{p}) \mod q $$

This simplifies to:

$$ k = \frac{h_1 - h_2}{s_1 - s_2} $$

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)